## --- [Day 17: Conway Cubes](https://adventofcode.com/2020/day/17) ---

During a cycle, all cubes simultaneously change their state according to the following rules:

* If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
* If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.



In [248]:
INPUT_FILE = 'input_d17.txt'

example_input = '''.#.
..#
###
'''

class CubeSpace:
    def __init__(self):
        self.cubes = {}
        self.neighbors = {}
        self.cycles_run = 0
        
    def load_state(self, state: str) -> None:
        """
        Takes initial input and loads it into state
        :param state: string
        """
        lines = state.split()
        for y in range(len(lines)):
            line = lines[y].rstrip()
            for x in range(len(line)):
                self.set_cube(x, y, 0, line[x] == '#')
            
    def set_cube(self, x: int, y: int, z:int, active:bool) -> None:
        """
        Sets the state of a single cube
        :param x: x-coordinate
        :param y: y-coordinate
        :param z: z-coordinate
        :param active: cube state
        """
        # Update the neighbor counts as necessary
        if active and ((x, y, z) not in self.cubes or not self.cubes[(x,y,z)]):
            # Addding previously unknown OR activating inactive cube; increment neighbors
            self.update_neighbors(x, y, z, 1)
        elif not active and (x, y, z) in self.cubes and self.cubes[(x, y, z)] :
            # Inactivating a previously active cube, decrement neighbors
            self.update_neighbors(x, y, z, -1)
            
        # Update the cube state
        self.cubes[(x, y, z)] = active
                        
        
    def update_neighbors(self, x: int, y: int, z: int, inc: int) -> None:
        """
        Increments or decrements the count of neighbors for the 26 cubes adjacent
        to the one specified.
        :param x: x-coordinate of active cube
        :param y: y-coordinate of active cube
        :param z: z-coordinate of cube
        :param inc: increment value
        """            
        for xn in [x-1, x, x+1]:
            for yn in [y-1, y, y+1]:
                for zn in [z-1, z, z+1]:
                    # Skip the current cube itself
                    if not (x == xn and y == yn and z == zn):
                        if (xn, yn, zn) in self.neighbors:
                            self.neighbors[(xn, yn, zn)] += val
                        elif val == 1:
                            self.neighbors[(xn, yn, zn)] = 1
                        else:
                            raise ValueError('Tried to decrement empty neighbor!')
                                        
    def run_cycle(self, doIt=True) -> None:
        '''
        Updates state of every known cube based on number of active
        adjacent neighbors as follows:
        1. If a cube is active and exactly 2 or 3 of its neighbors are also active,
            the cube remains active. Otherwise, the cube becomes inactive.
        2. If a cube is inactive but exactly 3 of its neighbors are active, the
            cube becomes active. Otherwise, the cube remains inactive.
        '''
        # First, identify which cubes will change state
        cube_updates = []
        
        # Find active cubes that need to be inactivated
        for coordinates, cube in self.cubes.items():
            if cube and self.neighbors[coordinates] not in [2,3]:
                cube_updates.append((coordinates, False))
                
        # Find inactive cubes that will be activated (some may not be in self.cubes yet!)
        for coordinates, num_neighbors in self.neighbors.items():
            if num_neighbors == 3 and (coordinates not in self.cubes or not self.cubes[coordinates]):
                cube_updates.append((coordinates, True))
                
        for coordinates, active in cube_updates:
            self.set_cube(*coordinates, active)

        self.cycles_run += 1
            
    def run_cycles(self, num_cycles: int) -> None:
        '''
        Runs the specified number of cycles
        '''
        for _ in range(num_cycles):
            self.run_cycle()
        
    def count_active(self) -> int:
        """
        Counts the total number of active cubes
        """
        count = 0
        for _, val in self.cubes.items():
            if val:
                count += 1
                
        return count
    
    def print_state(self) -> None:
        '''
        Prints the internal state using the format in
        the puzzle description
        '''
        if self.cycles_run > 0:
            print(f'After {self.cycles_run} cycles:\n')
        else:
            print('Before any cycles:\n')
            
        min_z, max_z = 9999, 0
        min_y, max_y = 9999, 0
        min_x, max_x = 9999, 0
        
        for coordinates, active in self.cubes.items():
            if active:
                x, y, z = coordinates
                min_z, max_z = min(z, min_z), max(z, max_z)
                min_y, max_y = min(y, min_y), max(y, max_y)
                min_x, max_x = min(x, min_x), max(x, max_x)
            
        for z in range(min_z, max_z+1):
            print('z=', z)
            for y in range(min_y, max_y+1):
                line = ''
                for x in range(min_x, max_x+1):
                    if (x, y, z) in self.cubes and self.cubes[(x, y, z)]:
                        line += '#'
                    else:
                        line += '.'
                print(line)
            print('\n')


In [252]:
ex = CubeSpace()
ex.load_state(example_input)
assert ex.count_active() == 5
ex.print_state()
#ex.run_cycle()
#assert ex.count_active() == 11

Before any cycles:

z= 0
.#.
..#
###




In [255]:
ex.run_cycle()
ex.print_state()

After 1 cycles:

z= -1
#..
..#
.#.


z= 0
#.#
.##
.#.


z= 1
#..
..#
.#.




In [253]:
ex.count_active()


5

In [254]:
ex.neighbors

{(0, -1, -1): 1,
 (0, -1, 0): 1,
 (0, -1, 1): 1,
 (0, 0, -1): 1,
 (0, 0, 0): 1,
 (0, 0, 1): 1,
 (0, 1, -1): 3,
 (0, 1, 0): 3,
 (0, 1, 1): 3,
 (1, -1, -1): 1,
 (1, -1, 0): 1,
 (1, -1, 1): 1,
 (1, 0, -1): 2,
 (1, 0, 1): 2,
 (1, 1, -1): 5,
 (1, 1, 0): 5,
 (1, 1, 1): 5,
 (2, -1, -1): 1,
 (2, -1, 0): 1,
 (2, -1, 1): 1,
 (2, 0, -1): 2,
 (2, 0, 0): 2,
 (2, 0, 1): 2,
 (2, 1, -1): 4,
 (2, 1, 0): 3,
 (2, 1, 1): 4,
 (1, 0, 0): 1,
 (1, 2, -1): 4,
 (1, 2, 0): 3,
 (1, 2, 1): 4,
 (2, 2, -1): 3,
 (2, 2, 0): 2,
 (2, 2, 1): 3,
 (3, 0, -1): 1,
 (3, 0, 0): 1,
 (3, 0, 1): 1,
 (3, 1, -1): 2,
 (3, 1, 0): 2,
 (3, 1, 1): 2,
 (3, 2, -1): 2,
 (3, 2, 0): 2,
 (3, 2, 1): 2,
 (-1, 1, -1): 1,
 (-1, 1, 0): 1,
 (-1, 1, 1): 1,
 (-1, 2, -1): 1,
 (-1, 2, 0): 1,
 (-1, 2, 1): 1,
 (-1, 3, -1): 1,
 (-1, 3, 0): 1,
 (-1, 3, 1): 1,
 (0, 2, -1): 2,
 (0, 2, 1): 2,
 (0, 3, -1): 2,
 (0, 3, 0): 2,
 (0, 3, 1): 2,
 (1, 3, -1): 3,
 (1, 3, 0): 3,
 (1, 3, 1): 3,
 (0, 2, 0): 1,
 (2, 3, -1): 2,
 (2, 3, 0): 2,
 (2, 3, 1): 2,
 (3, 3, -1): 1,
